# Effective countermeasures against a group of interest

This notebook is intended to demonstrate potential uses of SATRAP for the investigation of CTI.

## Preliminaries

The data model of SATRAP is aligned to STIX 2.1, therefore, in the analysis we use the STIX2.1 vocabulary.
We also make use of concepts in the MITRE ATT&CK knowledge base.

The specifications of [ATT&CK in STIX 2.1](https://github.com/mitre-attack/attack-stix-data/blob/master/USAGE.md#the-attck-spec) describe the representation of the following ATT&CK concepts with STIX objects:

- Group: `intrusion-set`
- Mitigation: `course-of-action`
- Technique: `attack-pattern`

## Introduction

The **objective** of this investigation is to identify effective mitigations against techniques used by a specific group of interest.

In this example we assume that the interested organization is a Hospital in Luxembourg. Then, we define the following questions.

### Analysis questions
1. Is the intrusion set "Orangeworm" a potential threat for the organization?
2. In such a case, can we identify relevant courses of action to effectively mitigate the threat?


## Prerequisites
Load MITRE ATT&CK Enterprise using the ETL CLI of SATRAP as explained in the [user manual](/docs/manual/interfaces.md).

## Preparation
Set up the DB parameters and import the toolkit of SATRAP.

In [3]:
from satrap.service.satrap_analysis import CTIanalysisToolbox
from satrap.settings import TYPEDB_SERVER_ADDRESS, DB_NAME

satrap = CTIanalysisToolbox(TYPEDB_SERVER_ADDRESS, DB_NAME)

## Investigation
First, we do a quick search of [Orangeworm in ATT&CK](https://attack.mitre.org/groups/G0071/) to get an overview of the intrusion set. 

Highlights:
- Orangeworm has targeted the healthcare sector in the United States, Europe, and Asia since at least 2015.
- ID of Orangeworm in ATT&CK: `G0071`.

**Answer 1:** We realize that G0071 poses indeed a potential risk for the hospital and thus further investigation is required.

From now on, we can use the ID of Orangeworm to run some automated analysis operations.

### General information of Orangeworm

We start by getting an overview of the group in our CTI SKB.

In [4]:
try:
	t1 = satrap.get_attck_concept_info("G0071")
	print("Group: G0071")
	for data in t1:
		print(CTIanalysisToolbox.tabulate_stix_obj(data))
except Exception as error:
	print(f"{type(error).__name__}: {error}")

Group: G0071
+---------------------+---------------------------------------------------------+
| Property            | Value                                                   |
+=====================+=========================================================+
| type                | intrusion-set                                           |
+---------------------+---------------------------------------------------------+
| created             | 2018-10-17 00:14                                        |
+---------------------+---------------------------------------------------------+
| alias               | Orangeworm                                              |
+---------------------+---------------------------------------------------------+
| stix-id             | intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1     |
+---------------------+---------------------------------------------------------+
| modified            | 2024-04-10 21:33                                        |
+--

### What techniques does "G0071" use?

To evaluate potential vulnerabilities, we try to get information concerning the techniques used by Orangeworm.

In [5]:
techniques = satrap.techniques_used_by_group("G0071")
display(techniques)

{'attack-pattern--4f9ca633-15c5-463c-9724-bdcd54fde541': 'SMB/Windows Admin Shares',
 'attack-pattern--df8b2a25-8bdf-4856-953c-a04372b1c161': 'Web Protocols'}

We already knew about those techniques as they are directly listed in the previous ATT&CK link. So, we try to learn something more by activating the inference engine of SATRAP.

In [6]:
techniques = satrap.techniques_used_by_group("G0071", infer=True)
display(techniques)

{'attack-pattern--3e6831b2-bf4c-4ae6-b328-2e7c6633b291': 'Additional Local or Domain Groups',
 'attack-pattern--5bfccc3f-2326-4112-86cc-c1ece9d8a2b5': 'Binary Padding',
 'attack-pattern--3ccef7ae-cb5e-48f6-8302-897105fbf55c': 'Deobfuscate/Decode Files or Information',
 'attack-pattern--21875073-b0ee-49e3-9077-1e2a885359af': 'Domain Account',
 'attack-pattern--7610cada-1499-41a4-b3dd-46467b68d177': 'Domain Account',
 'attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c': 'Domain Groups',
 'attack-pattern--0d91b3c0-5e50-47c3-949a-2a796f04d144': 'Encrypted/Encoded File',
 'attack-pattern--f24faf46-3b26-4dbb-98f2-63460498e433': 'Fallback Channels',
 'attack-pattern--d63a3fb8-9452-4e9d-a60a-54be68d5998c': 'File Deletion',
 'attack-pattern--7bc57495-ea59-4380-be31-a64af124ef18': 'File and Directory Discovery',
 'attack-pattern--e6919abc-99f9-4c6c-95a5-14761e7b2add': 'Ingress Tool Transfer',
 'attack-pattern--bf90d72c-c00b-45e3-b3aa-68560560d4c5': 'Lateral Tool Transfer',
 'attack-pattern--2

SATRAP has indeed shown a larger list of techniques. And evidently we would like to find out the reason for these techniques to be associated with Orangeworm.

### Analyzing the association of techniques

SATRAP can provide the explanation for all the techniques displayed in one go, however, we decide to go one by one to simplify the analysis and to prioritize the techniques that we consider more likely to be potential threats to our organization.

For instance, we might be intrigued about why the technique $\mathsf{"Windows\ Command\ Shell"}$ forms part of the list above, assuming that the organization's systems run primarily on Windows OS.

In [7]:
# explain all techniques
# reason = satrap.explain_techniques_used_by_group("G0071")
reason = satrap.explain_techniques_used_by_group("G0071", "Windows Command Shell")
for tech in reason.explanations:
	print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")

Data inferred for query statement: { $usage (used:$technique, user:$group); $usage isa uses; }
Applied rule: transitive-use
Condition met: 
    use_yz: relationship--fb60b027-facd-4be2-b8b2-0fb9351ea235
    name_y: cmd
    use_xy: relationship--a8f41a5a-b6bd-4446-8f9d-22d0e7b4af74
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1)
    y: (cmd, tool--bba595da-b73a-4354-aa6c-224d4de7cb4e)
    name_z: Windows Command Shell
    name_x: Orangeworm
    z: (Windows Command Shell, attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62)
Inferred conclusion: 
    z: (Windows Command Shell, attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62)
    indirectly-uses:used: _RoleType[label: uses:used]
    indirectly-uses: _RelationType[label: indirectly-uses]
    _: _Relation[indirectly-uses:0x8470804d7ffffffffffffff5] (Inferred)
    indirectly-uses:indirect-user: _RoleType[label: indirectly-uses:indirect-user]
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1

In [8]:
import resources.ipynb_utils as utils
utils.info_explanation()

As per the note above, the explanation output can be interpreted as follows:
- The application of the inference rule called "transitive-use" resulted in the derivation of data satisfying the statement `{ $use (used:$technique, user:$group); $use isa uses; }` from the underlying query.
- 'Orangeworm uses cmd' and 'cmd uses Windows Command Shell' are relations in the CTI SKB that meet the conditions for the application of "transitive-use".
- The relation 'Orangeworm indirectly-uses Windows Command Shell' is derived as a result of applying the referred rule.


### Mitigating "Windows command shell"

Using the information that Orangeworm uses the technique "Windows Command Shell" via the tool "cmd", we first verify that "cmd" refers to the Windows CLI and then search for mititgations for this technique in SATRAP's knowledge base. To run the concerning SATRAP functions, we obtain the STIX ids from the explanation shown above.

In [9]:
cmd = satrap.search_stix_object("tool--bba595da-b73a-4354-aa6c-224d4de7cb4e")
if cmd:
	print(CTIanalysisToolbox.tabulate_stix_obj(cmd))

+---------------------+--------------------------------------------------------+
| Property            | Value                                                  |
+=====================+========================================================+
| type                | tool                                                   |
+---------------------+--------------------------------------------------------+
| created             | 2017-05-31 21:33                                       |
+---------------------+--------------------------------------------------------+
| stix-id             | tool--bba595da-b73a-4354-aa6c-224d4de7cb4e             |
+---------------------+--------------------------------------------------------+
| modified            | 2022-10-13 20:24                                       |
+---------------------+--------------------------------------------------------+
| spec-version        | 2.1                                                    |
+---------------------+-----

In [10]:
mitigations = satrap.mitigations_for_technique("attack-pattern--d1fcf083-a721-4223-aedf-bf8960798d62")
display(mitigations)

{'course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db': 'Execution Prevention'}

At this point, we have identified a suggested course of action (detailed below) to address a specific threat posed by the group that we consider relevant.

**Answer 2 (partial):** 'course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db': 'Execution Prevention'


In [11]:
mitigation = satrap.search_stix_object("course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db")
if mitigation:
	print(CTIanalysisToolbox.tabulate_stix_obj(mitigation))

+---------------------+---------------------------------------------------------+
| Property            | Value                                                   |
+=====================+=========================================================+
| type                | course-of-action                                        |
+---------------------+---------------------------------------------------------+
| created             | 2019-06-11 16:35                                        |
+---------------------+---------------------------------------------------------+
| stix-id             | course-of-action--47e0e9fe-96ce-4f65-8bb1-8be1feacb5db  |
+---------------------+---------------------------------------------------------+
| modified            | 2024-10-17 18:53                                        |
+---------------------+---------------------------------------------------------+
| spec-version        | 2.1                                                     |
+---------------

#### Why is "Domain Groups" listed as a technique used by Orangeworm?

More than one explanation path can lead to the association of a technique with a group, as in the case of $\mathsf{Domain\ Groups}$ for Orangeworm.

In [12]:
dg_explanation = satrap.explain_techniques_used_by_group("G0071", "Domain Groups")
for tech in dg_explanation.explanations:
	print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")

Data inferred for query statement: { $usage (used:$technique, user:$group); $usage isa uses; }
Applied rule: transitive-use
Condition met: 
    name_y: Kwampirs
    z: (Domain Groups, attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c)
    name_z: Domain Groups
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1)
    use_yz: relationship--040ff1f7-493d-4762-a9bb-db143a971464
    use_xy: relationship--36aad32e-5300-4db0-b5a2-8b7442123db1
    name_x: Orangeworm
    y: (Kwampirs, malware--c2417bab-3189-4d4d-9d60-96de2cdaf0ab)
Inferred conclusion: 
    indirectly-uses: _RelationType[label: indirectly-uses]
    _: _Relation[indirectly-uses:0x8470804d7ffffffffffffffe] (Inferred)
    x: (Orangeworm, intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1)
    indirectly-uses:indirect-user: _RoleType[label: indirectly-uses:indirect-user]
    z: (Domain Groups, attack-pattern--2aed01ad-3df3-4410-a8cb-11ea4ded587c)
    indirectly-uses:used: _RoleType[label: uses:used]
Mapping o

For cases with multiple explanations, the textual explanation can be further clarified using the interactive Graph viewer of TypeDB Studio as explained in the [... section]() of SATRAP's user manual.

We can obtain the query to execute in TypeDB Studio from the explanation result above:

In [13]:
print(dg_explanation.query)

match
$exref isa external-reference, has source-name 'mitre-attack';
$exref has external-id 'G0071';
$group isa intrusion-set;
$group has revoked false;
(referrer:$group, referenced:$exref) isa external-referencing;
$technique isa attack-pattern, has stix-id $id, has name 'Domain Groups';
$usage (used: $technique, user: $group) isa uses;
get;


The next figure shows the explanation paths in TypeDB Studio detailing why "Domain Gropus" is a technique used by "G0071"; the relation "indirectly-uses" framed in green corresponds to the inferred relation. These paths correspond to the textual explanations previously displayed.

<p align="center">
	<img src="./resources/figs/tech-g0071-paths.png" alt="Explanation traces in TypeDB Studio" width="90%">
</p>

In this graph, we can clearly see that the conclusion about the intrusion set "Orangeworm" using the attack pattern "Domain Groups" comes from the fact that it relies on the use of both, a tool called "Net" and the malware-family "Kwampirs", each of which in turn make use of "Domain Groups". 

### Which known mitigations are potentially relevant against "G0071"?

Finally, to address more generally the initial analysis question 2, we are interested in identifying a set of mitigations that can be associated with Orangeworm because they address one of more of the techniques used by this group. This can be achieved as follows:

In [14]:
display(satrap.related_mitigations(group_name="Orangeworm"))

{'course-of-action--12241367-a8b7-49b4-b86e-2236901ba50c': 'Network Intrusion Prevention',
 'course-of-action--9bb9e696-bff8-4ae1-9454-961fc7d91d5f': 'Privileged Account Management',
 'course-of-action--1dcaeb21-9348-42ea-950a-f842aaf1ae1f': 'Limit Access to Resource Over Network',
 'course-of-action--20f6a9df-37c4-4e20-9e47-025983b1b39d': 'Filter Network Traffic',
 'course-of-action--90c218c3-fbf8-4830-98a7-e8cfb7eaa485': 'Password Policies',
 'course-of-action--49961e75-b493-423a-9ec7-ac2d6f55384a': 'Password Policy Discovery Mitigation',
 'course-of-action--684feec3-f9ba-4049-9d8f-52d52f3e0e40': 'System Network Configuration Discovery Mitigation',
 'course-of-action--c1676218-c16a-41c9-8f7a-023779916e39': 'System Network Connections Discovery Mitigation',
 'course-of-action--82d8e990-c901-4aed-8596-cc002e7eb307': 'System Time Discovery Mitigation',
 'course-of-action--2f316f6c-ae42-44fe-adf8-150989e0f6d3': 'Operating System Configuration',
 'course-of-action--2ace01f8-67c8-43eb-b7b1

As before, we can get an explanation for why each of the listed mitigations are related to Orangeworm. For instance, we select to get an explanation for "Restrict File and Directory Permissions".

In [15]:
try:
	rel_explanation = satrap.explain_if_related_mitigation("G0071", "course-of-action--987988f0-cf86-4680-a875-2f6456ab2448")
	print(f"EXECUTED QUERY:\n{rel_explanation.query}\n")
	print("EXPLANATIONS:")
	for tech in rel_explanation.explanations:
		print(CTIanalysisToolbox.format_dict(tech.as_json()), end="\n")
except ValueError as err:
	print(err)

EXECUTED QUERY:
match
$intrusion-set isa intrusion-set, has stix-id 'intrusion-set--5636b7b3-d99b-4edd-aa05-ee649c1d4ef1', has name $group-name;
$course-of-action isa course-of-action, has stix-id 'course-of-action--987988f0-cf86-4680-a875-2f6456ab2448', has name $mitigation-name;
$r (source-role: $course-of-action, target-role: $intrusion-set) isa related-to;
get;

EXPLANATIONS:
Data inferred for query statement: { $r (source-role:$course-of-action, target-role:$intrusion-set); $r isa related-to; }
Applied rule: course-of-action-for-intrusion-set
Condition met: 
    mitigation: relationship--d107e5fd-53fc-478d-9040-85087070d7ff
    use: _Relation[indirectly-uses:0x8470804d7ffffffffffffde9] (Inferred)
    sdo: (Service Execution, attack-pattern--f1951e8a-500e-4a26-8803-76d95c4554b4)
    sdo_name: Service Execution
    course-of-action: (Restrict File and Directory Permissions, course-of-action--987988f0-cf86-4680-a875-2f6456ab2448)
    intrusion-set: (Orangeworm, intrusion-set--5636b7b

And again, we can run the printed query in TypeDB Studio to visualize an interactive explanation.